# Power Consumption Dashboard

In this notebook, we will explain how to build a little more complicated dashboard.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 10)
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output

## 1. Getting and exploring the data

In [2]:
# Power Consumption of Tetouan City Data Set
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00616/Tetuan%20City%20power%20consumption.csv')
print(df.shape)
df.head()

(52416, 9)


,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [3]:
# Renaming some columns
df.rename(columns={'Zone 1 Power Consumption': 'Zone1', 'Zone 2  Power Consumption': 'Zone2', 'Zone 3  Power Consumption': 'Zone3'}, inplace=True)
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [4]:
df.DateTime = pd.to_datetime(df.DateTime)
df.dtypes

DateTime                 datetime64[ns]
Temperature                     float64
Humidity                        float64
Wind Speed                      float64
general diffuse flows           float64
diffuse flows                   float64
Zone1                           float64
Zone2                           float64
Zone3                           float64
dtype: object

In [5]:
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3
0,2017-01-01 00:00:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,2017-01-01 00:10:00,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,2017-01-01 00:20:00,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,2017-01-01 00:30:00,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,2017-01-01 00:40:00,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [6]:
# Getting one value per day
dfd = df.groupby(pd.Grouper(key='DateTime', freq='1D')).mean().reset_index()
print(dfd.shape)
dfd.head()

(364, 9)


,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3
0,2017-01-01,9.675299,68.519306,0.315146,121.390771,25.993924,28465.232067,17737.791287,17868.795181
1,2017-01-02,12.476875,71.456319,0.076563,120.404486,27.227410,28869.493671,19557.725431,17820.763053
2,2017-01-03,12.100000,74.981667,0.076715,120.686014,28.574660,30562.447257,20057.269504,17620.803213
3,2017-01-04,10.509479,75.459792,0.082417,122.959319,28.827222,30689.831224,20102.077001,17673.694779
4,2017-01-05,10.866444,71.040486,0.083896,118.749861,29.741437,30802.911393,20033.941237,17664.176707


In [7]:
# Creating the Month variable
dfd['Month'] = dfd.DateTime.dt.month
dfd.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3,Month
0,2017-01-01,9.675299,68.519306,0.315146,121.390771,25.993924,28465.232067,17737.791287,17868.795181,1
1,2017-01-02,12.476875,71.456319,0.076563,120.404486,27.227410,28869.493671,19557.725431,17820.763053,1
2,2017-01-03,12.100000,74.981667,0.076715,120.686014,28.574660,30562.447257,20057.269504,17620.803213,1
3,2017-01-04,10.509479,75.459792,0.082417,122.959319,28.827222,30689.831224,20102.077001,17673.694779,1
4,2017-01-05,10.866444,71.040486,0.083896,118.749861,29.741437,30802.911393,20033.941237,17664.176707,1


In [8]:
dfd.tail()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3,Month
359,2017-12-26,11.621840,69.070903,0.083062,106.836687,50.760389,30013.476975,24757.006239,11395.678272,12
360,2017-12-27,15.232917,59.445903,0.082028,95.890674,41.705062,29966.962399,25057.481845,11498.279312,12
361,2017-12-28,13.662361,62.839375,0.081354,112.620160,30.209361,30267.004647,24585.251099,11167.026811,12
362,2017-12-29,12.990486,49.078750,0.078181,122.220021,23.971021,29209.632446,24610.821315,11171.268507,12
363,2017-12-30,11.688993,51.361667,0.078174,123.814125,24.039944,28145.669624,24211.286693,11560.824329,12


In [9]:
# Selecting January
dfd_sel = dfd[dfd['Month'] == 1]
dfd_sel.shape

(31, 10)

In [10]:
# Selecting two zones
zones = ['Zone1','Zone3']
zones

['Zone1', 'Zone3']

In [11]:
# Scatterplot of Power consumption vs Temperature
px.scatter(dfd_sel, x='Temperature', y=zones, 
            hover_data={'Temperature':':.2f', 'value':':.2f'},
            width=800, title='Temperature vs Power Consumption')

In [12]:
# Scatterplot of Power consumption vs Humidity
px.scatter(dfd_sel, x='Humidity', y=zones, 
            hover_data={'Humidity':':.2f', 'value':':.2f'},
            width=800, title='Humidity vs Power Consumption')

In [13]:
# Scatterplot of Power consumption vs Wind Speed
px.scatter(dfd_sel, x='Wind Speed', y=zones, 
            hover_data={'Wind Speed':':.2f', 'value':':.2f'},
            width=800, title='Wind Speed vs Power Consumption')

## 2. Building the layout

In [14]:
# Creating the JupyterDash app
app = JupyterDash()

In [15]:
# Defining the title
html.H1('Power consumption in Tetouan, 2017')

H1('Power consumption in Tetouan, 2017')

In [16]:
# Adding some style to the title
html.Div(html.H1('Power consumption in Tetouan, 2017'), 
            style={'text-align': 'center', 'color':'grey'})

Div(children=H1('Power consumption in Tetouan, 2017'), style={'text-align': 'center', 'color': 'grey'})

In [17]:
# Defining a checklist to select the zones
dcc.Checklist(id='zone-check',
            options={
                'Zone1': ' Zone 1 ',
                'Zone2': ' Zone 2 ',
                'Zone3': ' Zone 3 ',                
            }, 
            value=['Zone1', 'Zone2', 'Zone3'],
        ), # end of checklist

(Checklist(options={'Zone1': ' Zone 1 ', 'Zone2': ' Zone 2 ', 'Zone3': ' Zone 3 '}, value=['Zone1', 'Zone2', 'Zone3'], id='zone-check'),)

In [18]:
# Defining a slider to select a month
dcc.Slider(1, 12, 1, value=1,
            id='month-slider',
            marks={
                1: 'Jan', 
                2: 'Feb', 
                3: 'Mar', 
                4: 'Apr', 
                5: 'May', 
                6: 'Jun', 
                7: 'Jul', 
                8: 'Aug', 
                9: 'Sep', 
                10: 'Oct', 
                11: 'Nov', 
                12: 'Dec'
            },
            included=False,
            tooltip={"placement": "top", "always_visible": True}
        ), # end of slider

(Slider(min=1, max=12, step=1, marks={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}, value=1, included=False, tooltip={'placement': 'top', 'always_visible': True}, id='month-slider'),)

We want to place the `checklist` and the `slider` horizontally side by side. To do it, we add the `stile` option with `'width': '20%'` for the checklist, and `'width': '77%'` for the slider. For both we have to include `'display':'inline-block'`.

In [19]:
# Adding some style to the checklist
dcc.Checklist(id='zone-check',
    options={
        'Zone1': ' Zone 1    ',
        'Zone2': ' Zone 2    ',
        'Zone3': ' Zone 3    ',                
    }, 
    value=['Zone1', 'Zone2', 'Zone3'],
    style={'width':'20%', 'display':'inline-block', 'horizontal-align':'right'}    
), # end of checklist

(Checklist(options={'Zone1': ' Zone 1    ', 'Zone2': ' Zone 2    ', 'Zone3': ' Zone 3    '}, value=['Zone1', 'Zone2', 'Zone3'], style={'width': '20%', 'display': 'inline-block', 'horizontal-align': 'right'}, id='zone-check'),)

In [20]:
# Adding some style to the slider
html.Div([
        dcc.Slider(1, 12, 1, value=1,
            id='month-slider',
            marks={
                1: 'Jan', 
                2: 'Feb', 
                3: 'Mar', 
                4: 'Apr', 
                5: 'May', 
                6: 'Jun', 
                7: 'Jul', 
                8: 'Aug', 
                9: 'Sep', 
                10: 'Oct', 
                11: 'Nov', 
                12: 'Dec'
            },
            included=False,
            tooltip={"placement": "top", "always_visible": True}
        ), # end of slider
    ], style={'width': '77%', 'display': 'inline-block'}), 

(Div(children=[Slider(min=1, max=12, step=1, marks={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}, value=1, included=False, tooltip={'placement': 'top', 'always_visible': True}, id='month-slider')], style={'width': '77%', 'display': 'inline-block'}),)

In [21]:
# defining an horizontal line
html.Hr(),

(Hr(None),)

In [22]:
# Defining the scatterplot of Power consumption vs Temperature
dcc.Graph(id='fig-s1')

Graph(id='fig-s1')

In [23]:
# Defining the scatterplot of Power consumption vs Humidity
dcc.Graph(id='fig-s2')

Graph(id='fig-s2')

In [24]:
# scatterplot of Power consumption vs Wind Speed
dcc.Graph(id='fig-s3')  

Graph(id='fig-s3')

In [25]:
# Adding some style to the scatterplot
dcc.Graph(id='fig-s1', style={'width': '32%', 'display': 'inline-block'})

Graph(id='fig-s1', style={'width': '32%', 'display': 'inline-block'})

In [26]:
# Defining the scatterplot of Power consumption vs Humidity
dcc.Graph(id='fig-s2', style={'width': '32%', 'display': 'inline-block'})

Graph(id='fig-s2', style={'width': '32%', 'display': 'inline-block'})

In [27]:
# Defining the scatterplot of Power consumption vs Wind Speed
dcc.Graph(id='fig-s3', style={'width': '32%', 'display': 'inline-block'})

Graph(id='fig-s3', style={'width': '32%', 'display': 'inline-block'})

In [28]:
# Creating the layout
app.layout = html.Div([
    # title
    html.Div(html.H1('Power consumption in Tetouan, 2017'), 
            style={'text-align': 'center', 'color':'grey'}),
    # checklist        
    dcc.Checklist(id='zone-check',
        options={
            'Zone1': ' Zone 1    ',
            'Zone2': ' Zone 2    ',
            'Zone3': ' Zone 3    ',                
        }, 
        value=['Zone1', 'Zone2', 'Zone3'],
        style={'width':'20%', 'display':'inline-block', 'horizontal-align':'right'}    
    ), # end of checklist
    # slider
    html.Div([
        dcc.Slider(1, 12, 1, value=1,
            id='month-slider',
            marks={
                1: 'Jan', 
                2: 'Feb', 
                3: 'Mar', 
                4: 'Apr', 
                5: 'May', 
                6: 'Jun', 
                7: 'Jul', 
                8: 'Aug', 
                9: 'Sep', 
                10: 'Oct', 
                11: 'Nov', 
                12: 'Dec'
            },
            included=False,
            tooltip={"placement": "top", "always_visible": True},
        ), # end of slider
    ], style={'width': '67%', 'display': 'inline-block'}),    
    # horizontal line
    html.Hr(),
    # graphs
    dcc.Graph(id='fig-s1', style={'width': '28%', 'display': 'inline-block'}),
    dcc.Graph(id='fig-s2', style={'width': '28%', 'display': 'inline-block'}),
    dcc.Graph(id='fig-s3', style={'width': '28%', 'display': 'inline-block'}),    
])

Notice we placed the scatterplots side by side: `style={'width': '32%', 'display': 'inline-block'}`

## 3. Adding interactivity with callback functions

In [29]:
# Set up the callback function
@app.callback(
    Output('fig-s1',  'figure'),
    Output('fig-s2',  'figure'),
    Output('fig-s3',  'figure'),
    Input ('month-slider', 'value'),
    Input ('zone-check',   'value'),
)
def update_graphs(sel_month, sel_zones):
    # filtering the dataframe
    dfd_sel = dfd[dfd['Month'] == sel_month]

    # getting the zones to plot    
    zones = [z for z in ['Zone1','Zone2','Zone3'] if z in sel_zones]
    
    # Temperature vs Power Consumption
    fig_s1 = px.scatter(dfd_sel, x='Temperature', y=zones,
                hover_data={'Temperature':':.2f', 'value':':.2f'},
                title='<b> Temperature vs Power Consumption </b>')
    
    # Humidity vs Power Consumption
    fig_s2 = px.scatter(dfd_sel, x='Humidity', y=zones,
                hover_data={'Humidity':':.2f', 'value':':.2f'},
                title='<b> Humidity vs Power Consumption </b>')
    
    # Wind Speed vs Power Consumption
    fig_s3 = px.scatter(dfd_sel, x='Wind Speed', y=zones,
                hover_data={'Wind Speed':':.2f', 'value':':.2f'},
                title='<b> Wind Speed vs Power Consumption </b>')    
        
    return fig_s1, fig_s2, fig_s3

Notice we have two inputs: 
- the checklist for the zones: `Input (component_id='zone-check',   component_property='value')` and 
- the slider for a month: `Input (component_id='month-slider', component_property='value')`

and three outputs corresponding to the three scatterplots.

In [30]:
# filtering the dataframe for choosing a month
sel_month = 1   # January
dfd_sel = dfd[dfd['Month'] == sel_month]
dfd_sel.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3,Month
0,2017-01-01,9.675299,68.519306,0.315146,121.390771,25.993924,28465.232067,17737.791287,17868.795181,1
1,2017-01-02,12.476875,71.456319,0.076563,120.404486,27.227410,28869.493671,19557.725431,17820.763053,1
2,2017-01-03,12.100000,74.981667,0.076715,120.686014,28.574660,30562.447257,20057.269504,17620.803213,1
3,2017-01-04,10.509479,75.459792,0.082417,122.959319,28.827222,30689.831224,20102.077001,17673.694779,1
4,2017-01-05,10.866444,71.040486,0.083896,118.749861,29.741437,30802.911393,20033.941237,17664.176707,1


In [31]:
# getting the zones to plot    
sel_zones = "['Zone1','Zone3']"
zones = [z for z in ['Zone1','Zone2','Zone3'] if z in sel_zones]
zones

['Zone1', 'Zone3']

## 4. Running the dashboard

In [35]:
# Run app and display result inline in the notebook
app.run_server(mode='inline', port=8051)

The dashboard looks like this.

<img src="power-dash.gif" align="center">

You can run the dashboard using the python file `power_dash.py`. 

You need to made some changes:
- Use: `from dash import Dash` instead of `from jupyter_dash import JupyterDash`

- Use `app = Dash()` instead of `app = JupyterDash()` 

- For running the dashboard use the code:

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)

instead of:

In [ ]:
app.run_server(mode='inline', port=8051)

For running the dashboard, you need to combine all the code snippets into one single Python script. You have this code in the file: `power_dash.py`. You need to run it locally using the command: `python power_dash.py`

Remember Dash creates web applications. The `URL http://127.0.0.1:8050/` is the default address to access the app. You can go to it and see your first Python interactive dashboard opening in your browser!

Good Luck!!

## References

- https://dash.plotly.com/dash-core-components/checklist
- https://dash.plotly.com/dash-core-components/slider
- https://dash.plotly.com/layout
- https://dash.plotly.com/basic-callbacks
- Salam, A., & El Hibaoui, A. (2018) "Comparison of Machine Learning Algorithms for the Power Consumption Prediction: -Case Study of Tetouan city". In 2018 6th International Renewable and Sustainable Energy Conference (IRSEC) (pp. 1-5)